In [ ]:
#Using Azure
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [ ]:
def azure_upload_blob(connect_str, container_name, blob_name, data):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
  blob_client.upload_blob(data, overwrite=True)
  print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
  download_stream = blob_client.download_blob
  return download_stream.readall()

In [ ]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'vehiclecollisions'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

df_vehicle_collisions = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
  print(blob.name)
  blob_client = container_client.get_blob_client(blob=blob.name)
  blob_data=blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))
  # Display the head of the DataFrame
  print(df.shape)
  # since I have only one csv, I am doing the following instructions
  df_raw = df.copy()

vehicle_collisions.csv


<ipython-input-20-6e9e6ae215fb>:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


(2087324, 29)


In [ ]:
df_raw.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [ ]:
df_raw.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Select the columns from 'CRASH DATE' to 'VEHICLE TYPE CODE 5' by name
df_clean = df_raw.loc[:, 'CRASH DATE':'VEHICLE TYPE CODE 5']

In [ ]:
df_clean.shape

(2087324, 29)

In [ ]:
df_clean.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_clean.isnull().sum()/len(df_clean)

CRASH DATE                       0.000000
CRASH TIME                       0.000000
BOROUGH                          0.311162
ZIP CODE                         0.311284
LATITUDE                         0.112340
LONGITUDE                        0.112340
LOCATION                         0.112340
ON STREET NAME                   0.212682
CROSS STREET NAME                0.378738
OFF STREET NAME                  0.831572
NUMBER OF PERSONS INJURED        0.000009
NUMBER OF PERSONS KILLED         0.000015
NUMBER OF PEDESTRIANS INJURED    0.000000
NUMBER OF PEDESTRIANS KILLED     0.000000
NUMBER OF CYCLIST INJURED        0.000000
NUMBER OF CYCLIST KILLED         0.000000
NUMBER OF MOTORIST INJURED       0.000000
NUMBER OF MOTORIST KILLED        0.000000
CONTRIBUTING FACTOR VEHICLE 1    0.003295
CONTRIBUTING FACTOR VEHICLE 2    0.155433
CONTRIBUTING FACTOR VEHICLE 3    0.928470
CONTRIBUTING FACTOR VEHICLE 4    0.983836
CONTRIBUTING FACTOR VEHICLE 5    0.995617
COLLISION_ID                     0

In [ ]:
len(df_clean)

2087324

In [ ]:
df_cleanX = df_clean.drop(['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'], axis=1)

In [ ]:
df_cleanX.isnull().sum()/len(df_cleanX)

CRASH DATE                       0.000000
CRASH TIME                       0.000000
BOROUGH                          0.311162
ZIP CODE                         0.311284
LATITUDE                         0.112340
LONGITUDE                        0.112340
LOCATION                         0.112340
NUMBER OF PERSONS INJURED        0.000009
NUMBER OF PERSONS KILLED         0.000015
NUMBER OF PEDESTRIANS INJURED    0.000000
NUMBER OF PEDESTRIANS KILLED     0.000000
NUMBER OF CYCLIST INJURED        0.000000
NUMBER OF CYCLIST KILLED         0.000000
NUMBER OF MOTORIST INJURED       0.000000
NUMBER OF MOTORIST KILLED        0.000000
CONTRIBUTING FACTOR VEHICLE 1    0.003295
CONTRIBUTING FACTOR VEHICLE 2    0.155433
CONTRIBUTING FACTOR VEHICLE 3    0.928470
CONTRIBUTING FACTOR VEHICLE 4    0.983836
CONTRIBUTING FACTOR VEHICLE 5    0.995617
COLLISION_ID                     0.000000
dtype: float64

In [ ]:
df_cleanX.dropna(subset=['LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED'], inplace=True)
df_cleanX.head(5)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",0.0,0.0,0,...,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",0.0,0.0,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609
6,12/14/2021,17:05,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",0.0,0.0,0,...,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4486555
7,12/14/2021,8:17,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",2.0,0.0,0,...,0,0,2,0,Unspecified,Unspecified,NaN,NaN,NaN,4486660
8,12/14/2021,21:10,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",0.0,0.0,0,...,0,0,0,0,Driver Inexperience,Unspecified,NaN,NaN,NaN,4487074


In [ ]:
df_cleanX.isnull().sum()/len(df_cleanX)

CRASH DATE                       0.000000
CRASH TIME                       0.000000
BOROUGH                          0.243971
ZIP CODE                         0.244095
LATITUDE                         0.000000
LONGITUDE                        0.000000
LOCATION                         0.000000
NUMBER OF PERSONS INJURED        0.000000
NUMBER OF PERSONS KILLED         0.000000
NUMBER OF PEDESTRIANS INJURED    0.000000
NUMBER OF PEDESTRIANS KILLED     0.000000
NUMBER OF CYCLIST INJURED        0.000000
NUMBER OF CYCLIST KILLED         0.000000
NUMBER OF MOTORIST INJURED       0.000000
NUMBER OF MOTORIST KILLED        0.000000
CONTRIBUTING FACTOR VEHICLE 1    0.003382
CONTRIBUTING FACTOR VEHICLE 2    0.158570
CONTRIBUTING FACTOR VEHICLE 3    0.928838
CONTRIBUTING FACTOR VEHICLE 4    0.983744
CONTRIBUTING FACTOR VEHICLE 5    0.995523
COLLISION_ID                     0.000000
dtype: float64

In [ ]:
columns_to_check = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
                    'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
                    'CONTRIBUTING FACTOR VEHICLE 5']

# Use DataFrame.notna() to get a boolean DataFrame where True indicates non-NaN entries
# Then use DataFrame.sum(axis=1) to sum across rows (i.e., sum True values for each row)
df_cleanX['COUNT OF VEHICLES INVOLVED'] = df_cleanX[columns_to_check].notna().sum(axis=1)
df_cleanX.isnull().sum()/len(df_cleanX)

CRASH DATE                       0.000000
CRASH TIME                       0.000000
BOROUGH                          0.243971
ZIP CODE                         0.244095
LATITUDE                         0.000000
LONGITUDE                        0.000000
LOCATION                         0.000000
NUMBER OF PERSONS INJURED        0.000000
NUMBER OF PERSONS KILLED         0.000000
NUMBER OF PEDESTRIANS INJURED    0.000000
NUMBER OF PEDESTRIANS KILLED     0.000000
NUMBER OF CYCLIST INJURED        0.000000
NUMBER OF CYCLIST KILLED         0.000000
NUMBER OF MOTORIST INJURED       0.000000
NUMBER OF MOTORIST KILLED        0.000000
CONTRIBUTING FACTOR VEHICLE 1    0.003382
CONTRIBUTING FACTOR VEHICLE 2    0.158570
CONTRIBUTING FACTOR VEHICLE 3    0.928838
CONTRIBUTING FACTOR VEHICLE 4    0.983744
CONTRIBUTING FACTOR VEHICLE 5    0.995523
COLLISION_ID                     0.000000
COUNT OF VEHICLES INVOLVED       0.000000
dtype: float64